In [45]:
import pandas as pd
import re
import seaborn as sns
import numpy as np

In [46]:
data =pd.read_csv('laptop_data.csv')

In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        1303 non-null   int64  
 1   Company           1303 non-null   object 
 2   TypeName          1303 non-null   object 
 3   Inches            1303 non-null   float64
 4   ScreenResolution  1303 non-null   object 
 5   Cpu               1303 non-null   object 
 6   Ram               1303 non-null   object 
 7   Memory            1303 non-null   object 
 8   Gpu               1303 non-null   object 
 9   OpSys             1303 non-null   object 
 10  Weight            1303 non-null   object 
 11  Price             1303 non-null   float64
dtypes: float64(2), int64(1), object(9)
memory usage: 122.3+ KB


In [48]:
#İŞİMİZE YARAMAYACAK OLAN SÜTUNALRI ÇIKARIYORUZ.
data = data.drop(['Unnamed: 0'],axis=1)

In [49]:
#ScreenResolution SÜTUNUNDA IPS YAZANLAR İÇİN YENİ BİR SÜTUN OLUŞTURUP 1 VEYA 0 YAZIYORUZ.
def ıps_screen(text):
    if 'IPS' in text:
        return 1
    else:
        return 0
data['IPS_Screen'] = data['ScreenResolution'].apply(ıps_screen)

In [50]:
#ScreenResolution SÜTUNUNDA Touchscreen YAZANLAR İÇİN YENİ BİR SÜTUN OLUŞTURUP 1 VEYA 0 YAZIYORUZ.
def dokunmatik_ekran(text):
    if 'Touchscreen' in text  :
        return 1
    else:
        return 0
data['touchscreen'] = data['ScreenResolution'].apply(dokunmatik_ekran)

In [51]:
#ScreenResolution SÜTUNUNDA YAZAN WİDTH VE HEIGHT DEĞERLERİNİ ALMAK İÇİN SADECE SÜTUNDAKİ SAYILARI ALIYORUZ.
#ARDINDAN İLK 4 RAKAMI ALIP BUNLAR WİDTH SÜTUNUNA GERİ KALAN RAKAMLARI DA HEIGHT SÜTUNUNA VERİYORUZ.
def sayilari_bul(txt):
    txt = "".join(re.findall('\d',txt))
    return txt
data['ScreenResolution'] = data['ScreenResolution'].apply(sayilari_bul)
data['screen_width'] = data['ScreenResolution'].str[:4]
data['screen_height'] = data['ScreenResolution'].str[4:]

In [52]:
#CPU VE GPU SÜTUNLARINDA ÇOK FAZLA BİLGİ VAR BİZ SADECE MARKALARINI ALIP BAŞKA SÜTUNLARA YAZDIRIYORUZ.
def cpu_find(text):
    if 'Intel' in text:
        return 'Intel'
    elif 'AMD' in text:
        return 'AMD'
    else:
        return 'Samsung'
data['cpu_model'] = data['Cpu'].apply(cpu_find)


def gpu_find(text):
    if 'Intel' in text:
        return 'Intel'
    elif 'AMD' in text:
        return 'AMD'
    elif 'ARM' in text:
        return 'ARM'
    else:
        return 'Nvidia'
data['gpu_model'] = data['Gpu'].apply(gpu_find)


In [53]:
#GPU MODELİ İÇİN RTX VE GTX ÖNEMLİ BİR FAKTÖR. BUNKLARIN OLMASI FİYATI ETKİLER BU NEDENLE BUNUN İÇİNDE BİR SÜTUN YAPACAĞIZ.
data['gtx_rtx'] = data['Gpu'].str.contains('GTX|RTX').astype(int)

In [54]:
def cpu_level(cpu):
    if 'i3' in cpu: return 3
    elif 'i5' in cpu: return 5
    elif 'i7' in cpu: return 7
    elif 'i9' in cpu: return 9
    else: return 0
data['cpu_level'] = data['Cpu'].apply(cpu_level)

In [55]:
#RAM SÜTUNUNDA SADECE YAZAN SAYILARIM ALIP AYNI SÜTUNA ATIYORUZ
data['Ram'] = data['Ram'].apply(sayilari_bul) #BURADA YİNE YUKARIDA TANIMLADIĞIMIZ SAYILARI BUL FONKSİYONUNU KULLANDIK

In [56]:
#WEIGHT SÜTUNUNDA SADECE FLOAT ŞEKLİNDE OLAN SAYILARI ALIYORUZ. VE AYNI SÜTUNA ATIYORUZ.
def weight(kg):
    kg = re.findall(r"\d+\.?\d*",kg)
    return kg[0]
data['Weight'] = data['Weight'].apply(weight)

In [57]:
#MEMORY SÜTUNUNDA YER ALAN HAFIZA TİPİ VE BÜYÜKLÜĞÜNÜ AYRI AYRI SÜTUNLARDA YAZIYORUZ. YANİ HDD SSD FLASH VE HYBRID İÇİN AYRI SÜTUNLAR OLUŞTURUYORUZ.
#BUNLARIN ALTINA DA BÜYÜKLÜKLERİNİ YAZIYORUZ.
def hafiza_tipi(text):
    hdd=0
    ssd=0
    flash=0
    hybrid=0

    doc = text.split('+')
    for mm in doc:
        match = re.findall(r'\d+\.?\d*',mm)
        if not match:
            continue
        number = float(match[0])
        if 'TB' in mm:
            number = number*1024

        if 'SSD' in mm:
            ssd += number
        elif 'HDD' in mm:
            hdd += number
        elif 'Flash Storage' in mm:
            flash += number
        elif 'Hybrid' in mm:
            hybrid += number

    return pd.Series([hdd,ssd,flash,hybrid])
data[['HDD_GB', 'SSD_GB', 'Flash_GB','Hybrid_GB']] = data['Memory'].apply(hafiza_tipi)

In [58]:
def os_name(os):
    if 'Windows' in os:
        return 'Windows'
    elif 'Mac' in os or 'macOS' in os:
        return 'Mac'
    elif 'Linux' in os:
        return 'Linux'
    elif 'Android' in os:
        return 'Android'
    elif 'Chrome' in os:
        return 'Chrome'
    else:
        return 'No OS'

data['OpSys_clean'] = data['OpSys'].apply(os_name)

In [59]:
#HAFIZA TÜRLERİNİ ÖNEMİNE GÖRE GÜÇLENDİRİP(YANİ DEĞERLERİNİ ÇARPIP) TOPLAMLARINI YENİ SÜTUNA YAZDIK
data['total_storage'] = data['HDD_GB']*0.8 + data['SSD_GB']*1.5 + data['Flash_GB']*1.1 + data['Hybrid_GB']

In [60]:
#VERİDE OBJECT OLAN SÜTUNLARI SAYIYA ÇEVİRDİK
data['Ram']=data['Ram'].astype('int64')
data['Weight'] = data['Weight'].astype('float64')
data['screen_width']=data['screen_width'].astype('int64')
data['screen_height']=data['screen_height'].astype('int64')
data['Price'] = data['Price'].astype('int64')

In [61]:
#YENİ PPI DEĞERİ İÇİN BİR SÜTUN OLUŞTURDUK
data['ppi'] = (((data['screen_width']**2) + (data['screen_height']**2))**0.5)/data['Inches']

In [62]:
#RAM MİKTARINA GÖRE DÜŞÜK OLANA 1, ORTALAMA OLANA 2, YÜKSEK OLANA 3 VERDİK.
def ram_normalize(ram):
    if ram < 16:
        return 1
    elif 16 < ram < 32:
        return 2
    else:
        return 3
data['ram_normalize'] = data['Ram'].apply(ram_normalize)

In [63]:
#ARTIK GEREKSİZ SÜTUNLARI TEMİZLEYEBİLİRİZ.
data=data.drop(['ScreenResolution','Cpu','Memory','Gpu','OpSys'],axis=1)

In [64]:
data.sample(5)

,Company,TypeName,Inches,Ram,Weight,Price,IPS_Screen,touchscreen,screen_width,screen_height,cpu_model,gpu_model,gtx_rtx,cpu_level,HDD_GB,SSD_GB,Flash_GB,Hybrid_GB,OpSys_clean,total_storage,ppi,ram_normalize
1284,Lenovo,2 in 1 Convertible,14.0,4,1.80,33992,1,1,1920,1080,Intel,Intel,0,7,0.0,128.0,0.0,0.0,Windows,192.0,157.350512,1
535,Dell,Notebook,15.6,8,1.93,72940,0,0,1920,1080,Intel,Nvidia,0,7,0.0,256.0,0.0,0.0,Windows,384.0,141.211998,1
614,Dell,Notebook,15.6,4,1.93,46300,0,0,1366,768,Intel,Intel,0,5,500.0,0.0,0.0,0.0,Linux,400.0,100.454670,1
430,Samsung,Netbook,11.6,4,1.15,14332,0,0,1366,768,Intel,Intel,0,0,0.0,0.0,16.0,0.0,Chrome,17.6,135.094211,1
282,Lenovo,Gaming,15.6,8,2.50,44169,1,0,1920,1080,Intel,Nvidia,1,5,0.0,256.0,0.0,0.0,Windows,384.0,141.211998,1


In [65]:
#ARTIK DATAMIZ EĞİTİLMEYE HAZIR.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company        1303 non-null   object 
 1   TypeName       1303 non-null   object 
 2   Inches         1303 non-null   float64
 3   Ram            1303 non-null   int64  
 4   Weight         1303 non-null   float64
 5   Price          1303 non-null   int64  
 6   IPS_Screen     1303 non-null   int64  
 7   touchscreen    1303 non-null   int64  
 8   screen_width   1303 non-null   int64  
 9   screen_height  1303 non-null   int64  
 10  cpu_model      1303 non-null   object 
 11  gpu_model      1303 non-null   object 
 12  gtx_rtx        1303 non-null   int32  
 13  cpu_level      1303 non-null   int64  
 14  HDD_GB         1303 non-null   float64
 15  SSD_GB         1303 non-null   float64
 16  Flash_GB       1303 non-null   float64
 17  Hybrid_GB      1303 non-null   float64
 18  OpSys_cl

In [66]:
abs(data.corr(numeric_only=True)['Price'].sort_values(ascending=False))

Price            1.000000
Ram              0.743007
SSD_GB           0.670682
ram_normalize    0.605813
cpu_level        0.564593
screen_height    0.552809
screen_width     0.542356
ppi              0.474640
total_storage    0.373788
gtx_rtx          0.363580
IPS_Screen       0.252207
Weight           0.210370
touchscreen      0.191226
Inches           0.068196
Hybrid_GB        0.008011
Flash_GB         0.040511
HDD_GB           0.094856
Name: Price, dtype: float64

In [67]:
x_data = data.drop(['Price','Inches','Hybrid_GB','Flash_GB','HDD_GB'],axis=1)

In [68]:
#ARTIK SAYISAL OLMAYAN TÜM SÜTUNLARI SAYIYA ÇEVİRECEĞİZ.
x=pd.get_dummies(x_data)

In [69]:
y= data[['Price']]

In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",100)

from sklearn.linear_model import LinearRegression,SGDRegressor,Ridge,Lasso,ElasticNet
from sklearn.neighbors import KNeighborsRegressor, RadiusNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree, ExtraTreeRegressor
#pip install xgboost
from xgboost import XGBRegressor
from sklearn.svm import SVR

from sklearn.neural_network import MLPRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

def algo_test(x,y):
        #Bütün modelleri tanımlıyorum
        L=LinearRegression()
        R=Ridge()
        Lass=Lasso()
        E=ElasticNet()
        sgd=SGDRegressor()
        ETR=ExtraTreeRegressor()
        GBR=GradientBoostingRegressor()
        kn=KNeighborsRegressor()
        rkn=RadiusNeighborsRegressor(radius=1.0)
        ada=AdaBoostRegressor()
        dt=DecisionTreeRegressor()
        xgb=XGBRegressor()
        svr=SVR()
        mlp_regressor = MLPRegressor()

       
        
        algos=[L,R,Lass,E,sgd,ETR,GBR,ada,kn,dt,xgb,svr,mlp_regressor]
        algo_names=['Linear','Ridge','Lasso','ElasticNet','SGD','Extra Tree','Gradient Boosting',
                    'KNeighborsRegressor','AdaBoost','Decision Tree','XGBRegressor','SVR','mlp_regressor']
        x=MinMaxScaler().fit_transform(x)
        x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=.20,random_state=42)
        
        r_squared= []
        rmse= []
        mae= []
        
        #Hata ve doğruluk oranlarını bir tablo haline getirmek için bir dataframe oluşturuyorum
        result=pd.DataFrame(columns=['R_Squared','RMSE','MAE'],index=algo_names)
        
        
        for algo in algos:
            p=algo.fit(x_train,y_train).predict(x_test)
            r_squared.append(r2_score(y_test,p))
            rmse.append(mean_squared_error(y_test,p)**.5)
            mae.append(mean_absolute_error(y_test,p))
        
            

        #result adlı tabloya doğruluk ve hata oranlarımı yerleştiriyorum
        result.R_Squared=r_squared
        result.RMSE=rmse
        result.MAE=mae
        
       #oluşturduğum result tablosunu doğruluk oranına (r2_score) göre sıralayıp dönüyor
        rtable=result.sort_values('R_Squared',ascending=False)
        return rtable

In [71]:
algo_test(x,y)

,R_Squared,RMSE,MAE
XGBRegressor,0.821637,16036.757741,9259.235808
Gradient Boosting,0.800365,16966.114202,10397.271382
Lasso,0.764596,18423.463848,12675.649549
Linear,0.764234,18437.612517,12693.268947
Ridge,0.757061,18716.004603,12638.850291
AdaBoost,0.735046,19545.638023,12413.468199
SGD,0.729159,19761.589173,12683.512414
Decision Tree,0.722958,19986.511248,12219.510536
Extra Tree,0.702003,20728.621684,12430.262452
KNeighborsRegressor,0.605232,23858.082426,19764.886123


In [122]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=.20,random_state=1)
xgb=XGBRegressor()

In [123]:
model = xgb.fit(x_train,y_train)

In [124]:
tahmin = model.predict(x_test)

In [125]:
r2_score(y_test,tahmin)

0.8649123012101791

In [126]:
mean_squared_error(y_test,tahmin)**0.5

13732.509409230934

In [128]:
import pickle

In [129]:
with open('laptop_price_predict.pkl','wb') as f:
    pickle.dump(model,f)